重複を検出する。<br>
Idでソートしたときの前後との重複、時間でソートしたときの前後との重複をそれぞれ feature にする。

In [9]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from parse import *
import seaborn as sns

In [101]:
CHUNKSIZE = 100000

TRAIN_DATE        = "../../data/train_date.csv"
TRAIN_NUMERIC     = "../../data/train_numeric.csv"
TRAIN_CATEGORICAL = "../../data/train_categorical.csv"
TEST_DATE         = "../../data/test_date.csv"
TEST_NUMERIC      = "../../data/test_numeric.csv"
TEST_CATEGORICAL  = "../../data/test_categorical.csv"

SEED = 0
CHUNKSIZE = 50000
NROWS = 1200000
#NROWS = 10000

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'

TEST_START_TIME      = "../../data/test_StartEndTime.csv"
TRAIN_START_TIME     = "../../data/train_StartEndTime.csv"

    

In [102]:
df_train_numeric = pd.read_csv(TRAIN_NUMERIC, nrows=NROWS, dtype='float32')
df_test_numeric  = pd.read_csv(TEST_NUMERIC,  nrows=NROWS, dtype='float32')


In [103]:
df_train_numeric.columns

Index(['Id', 'L0_S0_F0', 'L0_S0_F2', 'L0_S0_F4', 'L0_S0_F6', 'L0_S0_F8',
       'L0_S0_F10', 'L0_S0_F12', 'L0_S0_F14', 'L0_S0_F16',
       ...
       'L3_S50_F4245', 'L3_S50_F4247', 'L3_S50_F4249', 'L3_S50_F4251',
       'L3_S50_F4253', 'L3_S51_F4256', 'L3_S51_F4258', 'L3_S51_F4260',
       'L3_S51_F4262', 'Response'],
      dtype='object', length=970)

In [104]:
df_train_numeric = df_train_numeric.astype({'Id':'int32'})
df_train_numeric = df_train_numeric.astype({'Response':'int32'})

df_test_numeric['Response'] = -1
df_test_numeric = df_test_numeric.astype({'Id':'int32'})
df_test_numeric = df_test_numeric.astype({'Response':'int32'})

df_numeric_all = pd.concat([df_train_numeric, df_test_numeric])


In [105]:
df_train_start_time = pd.read_csv(TRAIN_START_TIME, dtype='float32')
df_test_start_time  = pd.read_csv(TEST_START_TIME, dtype='float32')
df_train_start_time = df_train_start_time.astype({'Id':'int32'})
df_train_start_time = df_train_start_time.drop({'Response':'int32'}, axis=1)
df_test_start_time = df_test_start_time.astype({'Id':'int32'})
df_start_all = pd.concat([df_train_start_time, df_test_start_time])

df_numeric_all = pd.merge(df_numeric_all, df_start_all, how='left', on='Id')



In [106]:
del df_train_numeric
del df_test_numeric
del df_train_start_time
del df_test_start_time


In [107]:
df_numeric_all = df_numeric_all.sort_values('Id')
df_numeric_all.reset_index(inplace=True, drop=True)
df_numeric_all = df_numeric_all.fillna(0)
df_numeric_all.head(30)

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response,StartTime,EndTime
0,1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,792.719971,800.700012
1,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,1025.630005,1060.069946
2,3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,671.919983,711.080017
3,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,82.239998,87.290001
4,5,-0.016,-0.026,-0.033,-0.016,0.205,-0.157,0.000,0.008,0.087,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,255.449997,256.279999
5,6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1313.119995,1315.750000
6,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1618.699951,1624.420044
7,8,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,743.380005,770.280029
8,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1149.199951,1154.160034
9,10,0.030,-0.026,0.003,0.003,-0.013,0.116,-0.007,0.008,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,907.340027,911.289978


In [118]:
# ソートしてから、feature が前の列と全く同じものを検出する。
def make_coh_feature(df_org, sort_columns):
    df = df_org.sort_values(sort_columns).reset_index(drop=True)
    
    fname_coh1      = sort_columns[0] + '_num_coh' 
    fname_coh_prev1 = sort_columns[0] + '_num_coh_prev' 

    vec_cnt      = np.zeros(len(df))
    vec_coh      = np.zeros(len(df))
    vec_coh_prev = np.zeros(len(df))

    loopcnt = 0
    length = df_org.shape[1] - 4
    print(length)
    for index, row in df.iterrows():
        vec_cur  = row.values[1:1+length] 
        if 0 < index:
            dup_num = (vec_cur == vec_prev).sum()
            vec_cnt[index] = dup_num
            if dup_num > length - 6:
                vec_coh[index]        = 1
                vec_coh_prev[index-1] = 1 
        vec_prev = vec_cur
    plt.hist(vec_cnt, bins = 100, range=(600, 1000))
    plt.show()
#    df_ret = pd.DataFrame({'Id'           : df['Id'].values, 
#                           fname_coh1     : vec_coh, 
#                           fname_coh_prev1: vec_coh_prev})
    df_ret = pd.DataFrame( columns=['Id', fname_coh1, fname_coh_prev1])
    df_ret.loc[:, 'Id']            = df['Id'].values 
    df_ret.loc[:, fname_coh1]      = vec_coh
    df_ret.loc[:, fname_coh_prev1] = vec_coh_prev
    df_ret = df_ret.sort_values('Id')
    return df_ret


In [ ]:
df_id_dup_list = make_coh_feature(df_numeric_all, ['Id'])
df_s_time_dup_list = make_coh_feature(df_numeric_all, ['StartTime', 'Id'])
df_e_time_dup_list = make_coh_feature(df_numeric_all, ['EndTime', 'Id'])

968


In [ ]:
df_id_dup_list.head(100)

In [ ]:
df_s_time_dup_list.head(100)

In [ ]:
df_e_time_dup_list[df_e_time_dup_list['EndTime_coh_prev'] == 1.0]

In [ ]:
# 保存する。
df_feat = pd.DataFrame(df_numeric_all['Id']).sort_values('Id')
df_feat = pd.merge(df_feat, df_id_dup_list, on='Id', how='left')
df_feat = pd.merge(df_feat, df_s_time_dup_list, on='Id', how='left')
df_feat = pd.merge(df_feat, df_e_time_dup_list, on='Id', how='left')





In [ ]:
df_feat.shape

In [ ]:
df_feat.to_csv("../../data/Feature_Numeric_Duplicates.csv", index=False)

1
